In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torchvision import models,transforms
import Images_helper

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
model = models.vgg19(pretrained=True).features
model.to(device)

In [ ]:
for param in model.parameters():
    param.requires_grad_(False) #stop all parameters gradient calculation to stop learning

In [ ]:
content = Images_helper.load_image("Images/S.png",400,[0.5,0.5,0.5],[0.5,0.5,0.5])
plt.imshow(Images_helper.Tensor_to_Image(content,[0.5,0.5,0.5],[0.5,0.5,0.5]))

In [ ]:
style = Images_helper.load_image("Images/magritte.jpg",400,[0.5,0.5,0.5],[0.5,0.5,0.5])
plt.imshow(Images_helper.Tensor_to_Image(style,[0.5,0.5,0.5],[0.5,0.5,0.5]))

In [ ]:
#key : layer number from the model in third cell => value : string representing name
style_layers = {
          '0' : 'conv1_1',
          '5' : 'conv2_1',
          '10' : 'conv3_1',
          '19' : 'conv4_1',
          '28' : 'conv5_1' 
         } 

content_layers = {'21' : 'conv4_2'} 

def get_layer_features(image,layers):
    features = {}
    imgtmp = image
    for number, layer in model._modules.items():
        imgtmp = layer(imgtmp)
        if number in layers:
            features[layers[number]] = imgtmp
    return features

def get_gram_matrix(tensor):
    #batchsize always 1
    batchsize, depth, height, width = tensor.size()
    tensor = tensor.view(depth,height*width)
    Gmatrix = torch.mm(tensor,tensor.t())
    return Gmatrix

In [ ]:
def content_loss(content_image,target_image):
    return torch.mean((content_image-target_image)**2)
def style_loss(style_features,target_features,weights,a=1):
    loss = 0
    for layer in style_features:
        _,d,h,w = style_features[layer].shape
        loss += (weights[layer] * torch.mean((target_features[layer]-style_features[layer])**2))/(d*h*w)
    return loss * a

In [ ]:
#loss weights
content_weight = 1 #alpha 
style_weight = 1e6 #omega
style_layers_weights = {'conv1_1' : 1,
                        'conv2_1' : 1,
                        'conv3_1' : 0.25,
                        'conv4_1' : 0.25,
                        'conv5_1' : 0.25}

iterations = 2

#get content and style images features before loop cause they will never change
content = content.to(device)
style = style.to(device)
content_image_features = get_layer_features(content,content_layers)
style_image_features = get_layer_features(style,style_layers)
#calc style grams
style_grams = {layer : get_gram_matrix(style_image_features[layer]) for layer in style_image_features}


# create a third "target" image and prep it for change
# it is a good idea to start of with the target as a copy of our *content* image
# then iteratively change its style
target = content.clone().requires_grad_(True).to(device)

#optimizer
optimizer = optim.Adam([target],lr=0.01)
for i in range(iterations):
    target_content_features = get_layer_features(target , content_layers)
    
    target_style_features = get_layer_features(target,style_layers)
    target_grams = {layer : get_gram_matrix(target_style_features[layer]) for layer in target_style_features}
    
    #calc loss
    content_loss = content_loss(content_image_features,target_content_features)
    style_loss =  style_loss(style_grams,target_grams,style_layers_weights)
    
    total_loss = (style_loss * style_weight) + (content_loss * content_weight)
    
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
plt.imshow(Images_helper.Tensor_to_Image(target,[0.5,0.5,0.5],[0.5,0.5,0.5]))
plt.show()    